In [1]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from experiment import rollout, ReplayBuffer, Trajectory, segments_to_training, load_model, save_model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
env = gym.make('CartPole-v1')

In [18]:
rb = ReplayBuffer(200, 100)
avg_reward = rollout(episodes=1000, env=env, replay_buffer=rb, render=False)

print(f"Average Episode Reward: {avg_reward}")

Average Episode Reward: 21.74


In [19]:
loss_object = torch.nn.BCEWithLogitsLoss().to(device)

In [20]:
batch_size = 1024

In [21]:
def train_step(inputs, targets):
    optimizer.zero_grad()
    
    predictions = model(inputs)
    loss = loss_object(predictions, targets.unsqueeze(1))
    
    loss.backward()
    optimizer.step()
    
    return loss

In [22]:
loss_sum = 0
loss_count = 0

In [ ]:
epochs = 1000000
epoch, model, optimizer, loss = load_model(env, device, train=True)

for i in range(1, epochs+1):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(x, y)
    loss_sum += loss
    loss_count += 1
    
    if i % 1000 == 0:        
        mean_reward = rollout(100, env=env, model=model, sample_action=True, replay_buffer=rb, 
                              device=device)
        
        print(f"Average Episode Reward: {mean_reward}")        
        
    
    if i % 200 == 0:
        print(f'i: {i}, Loss: {loss_sum/loss_count}') #'\t Accuracy: {accuracy_m.result()}')
        save_model(i, model, optimizer, loss)

i: 200, Loss: 0.6515129208564758
i: 400, Loss: 0.6484588384628296
i: 600, Loss: 0.6454616785049438
i: 800, Loss: 0.6438441276550293
Average Episode Reward: 53.4
i: 1000, Loss: 0.6416881084442139
i: 1200, Loss: 0.6385904550552368
i: 1400, Loss: 0.6354051828384399
i: 1600, Loss: 0.6322858333587646
i: 1800, Loss: 0.6294998526573181
Average Episode Reward: 53.65
i: 2000, Loss: 0.6267819404602051
i: 2200, Loss: 0.6236968636512756
i: 2400, Loss: 0.6204634308815002
i: 2600, Loss: 0.6175605654716492
i: 2800, Loss: 0.6146635413169861
Average Episode Reward: 69.38
i: 3000, Loss: 0.6120092272758484
i: 3200, Loss: 0.6089345812797546
i: 3400, Loss: 0.6058717966079712
i: 3600, Loss: 0.6026895046234131
i: 3800, Loss: 0.5996562838554382
Average Episode Reward: 53.63
i: 4000, Loss: 0.5967664122581482
i: 4200, Loss: 0.5947983264923096
i: 4400, Loss: 0.5925896167755127
i: 4600, Loss: 0.5904895663261414
i: 4800, Loss: 0.5882615447044373
Average Episode Reward: 73.28
i: 5000, Loss: 0.5860218405723572
i: 52

In [6]:
rb.sample_command()

(43.06, 43.32223427058613)

In [11]:
cmd = (500, 500) #rb.sample_command()
env = gym.make('CartPole-v1')
e, model, _, l = load_model(train=False, env=env, device=device)
print(f"Loaded model at epoch: {e} with loss {l}")
mean_reward = rollout(episodes=3, env=env, model=model, sample_action=True, 
                      cmd=cmd,
                      render=True, device=device)


print(f"Average Episode Reward: {mean_reward}")

Loaded model at epoch: 400 with loss 0.6158408522605896
Average Episode Reward: 35.666666666666664


In [27]:
env.close()